In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from langgraph.graph import MessagesState, StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage
from langchain_groq import ChatGroq
from IPython.display import Image, display


In [ ]:
llm = ChatGroq(model_name="Gemma2-9b-It")

In [ ]:
tavily_tool = TavilySearchResults()

In [ ]:
@tool
def get_city_details(prompt):
    "Should do a web search to find the required city details"
    response = tavily_tool.invoke(prompt)
    return response

In [ ]:
tools = [get_city_details]

In [ ]:
model_with_tools = llm.bind_tools(tools)

In [ ]:
class CityDetails(BaseModel):
    """Respond to the user with this"""
    state_name: str = Field(description="State name of the city")
    state_capital: str = Field(description="State capital of the city")
    country_name: str = Field(description="Country name of the city")
    country_capital: str = Field(description="Country capital of the city")

In [ ]:
# inherit 'messages' key from MessagesState, which is a list of chat messages
class AgentState(MessagesState):
    # Final structured response from the agent
    final_response = CityDetails

In [ ]:
model_with_structured_output = llm.with_structured_output(CityDetails)

In [ ]:
def call_model(state: AgentState):
    print(f"This is 01 input from call model {state}")
    response = model_with_tools.invoke(state["messages"])
    print(f"This is 02 response from call model {response}")
    # we retruen a list, because this will get added to the existing list
    return {"messages": [response]}

In [ ]:
def should_continue(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]

    # if there is no function call, then we respond to user
    if not last_message.tool_calls:
        return "respond"
    # Otherwise if there is, we continue with the tool
    else:
        return "continue"

In [ ]:
def respond(state: AgentState):
    print(f"Here is 03 state from response {state}")
    response = model_with_structured_output.invoke([HumanMessage(content=state["messages"][-1].content)])
    # we return the final answer
    print(f"This is 04 response from respond {response}")
    return {"final_response": response}

In [ ]:
# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("llm", call_model)
workflow.add_node("tools", ToolNode(tools=tools))
workflow.add_edge("respond", respond)

# Set the entrypoint as 'agent'
# This means that this node is the first one called
workflow.set_entry_point("llm")

# We now add a conditional edge
workflow.add_conditional_edges(
    "llm",
    should_continue,
    {
        "continue": "tools",
        "respond": "respond"
    }
)

workflow.add_edge("tools", "llm")
workflow.add_edge("respond", END)

graph = workflow.compile()

In [ ]:
# View
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
answer = graph.invoke(input={"messages": [("human", "tell me about city details for Tehran")]})["final_response"]

In [ ]:
answer